In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
sns.set_style("white")

In [1]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import datasets

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import spacy

In [19]:
from sklearn.base import TransformerMixin
from sklearn.pipeline import Pipeline
import string
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score

In [12]:
df_test = pd.read_csv('https://raw.githubusercontent.com/zsgithub2/Hublot-project/main/Data/test_data.csv')

In [38]:
len(df_test)

1142

In [14]:
df_train = pd.read_csv('https://raw.githubusercontent.com/zsgithub2/Hublot-project/main/Data/training_data.csv')

In [21]:
sp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS
punctuations = string.punctuation

In [22]:
def spacy_tokenizer(sentence):
    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = sp(sentence)

    # Lemmatize each token and convert each token into lowercase
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Return preprocessed list of tokens
    return mytokens

In [16]:
df_train.head()

,id,keyword,location,text,target
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1


In [23]:
df_train['processed'] = df_train['text'].apply(spacy_tokenizer)

In [26]:
df_test['processed'] = df_test['text'].apply(spacy_tokenizer)

In [24]:
def converttostr(input_seq):
   # Join all the strings in list
   final_str = " ".join(input_seq)
   return final_str

In [25]:
df_train['processed'] = df_train['processed'].apply(converttostr)

In [27]:
df_test['processed'] = df_test['processed'].apply(converttostr)

In [40]:
X = df_train['processed']
y = df_train['target']
X_test = df_test['processed']

In [31]:
tfidf_vector = TfidfVectorizer(tokenizer=spacy_tokenizer)

In [33]:
classifier = LogisticRegression(solver="lbfgs", max_iter=1000, random_state=72)

# Create pipeline
pipe = Pipeline([('vectorizer', tfidf_vector),
                 ('classifier', classifier)])

# Fit model on training set
pipe.fit(X, y)

Pipeline(memory=None,
         steps=[('vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_patt...',
                                 tokenizer=<function spacy_tokenizer at 0x7f44856b0488>,
                                 use_idf=True, vocabulary=None)),
                ('classifier',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_

In [34]:
def evaluate(true, pred):
    precision = precision_score(true, pred, average='micro')
    recall = recall_score(true, pred, average='micro')
    f1 = f1_score(true, pred, average='micro')
    print(f"CONFUSION MATRIX:\n{confusion_matrix(true, pred)}")
    print(f"ACCURACY SCORE:\n{accuracy_score(true, pred):.4f}")
    print(f"CLASSIFICATION REPORT:\n\tPrecision: {precision:.4f}\n\tRecall: {recall:.4f}\n\tF1_Score: {f1:.4f}")

In [37]:
y_pred = pipe.predict(X)
y_pred

array([0, 1, 1, ..., 0, 1, 1])

In [39]:
evaluate(y, y_pred)

CONFUSION MATRIX:
[[3589  112]
 [ 601 2169]]
ACCURACY SCORE:
0.8898
CLASSIFICATION REPORT:
	Precision: 0.8898
	Recall: 0.8898
	F1_Score: 0.8898


In [41]:
y_pred2 = pipe.predict(X_test)

In [42]:
len(y_pred2)

1142

In [43]:
y_pred2

array([0, 0, 1, ..., 1, 0, 1])